Set-up as in grooming_mouseSubsetSelection

In [1]:
from database.database import Database
from data.constants import dbDetails, dbUser_Krista
from models.experiments import Experiment
from models.reviewers import Reviewer
from database.update_database.update_from_data_dirs import update_from_data_dirs

Database.initialize(**dbDetails, **dbUser_Krista)
experiment_name = 'grooming'
reviewer_name = 'Krista K'

experiment = Experiment.from_db(experiment_name=experiment_name)
reviewer = Reviewer.from_db(reviewer_fullname=reviewer_name)

update_from_data_dirs(experiment)

No mouse in the database with mouse number 7044
No mouse in the database with mouse number 7068
No mouse in the database with mouse number 7025
No mouse in the database with mouse number 7076
No mouse in the database with mouse number 747
No mouse in the database with mouse number 7035
No mouse in the database with mouse number 750


In order to select the sequences, I'll have to know how the session data is stored either:
    1. All videos are present and no prior processing has been completed; or
    2. Session folders have 'cut' folders inside of them, containing trials.

In the case of 2, we can easily collect a list of all trials and then sample one per session.

In the case of 1, we can randomly sample a numbers 1-30, and this provides the starting point
for looking for a grooming sequence.

Our list of mice for analysis is:
7169, 7043, 7014, 745, 7061, 7062, 7063, 7064, 7065, 7166

First, the mouse details will be loaded from the ParticipantDetails folder in order to obtain a
list of all sessions for this animal.

In [6]:
from random import randrange
from pathlib import Path

from models.mouse import Mouse
from models.participant_details import ParticipantDetails
from models.sessions import Session

all_eartags = [7169, 7043, 7014, 745, 7061, 7062, 7063, 7064, 7065, 7166]

all_selected_sessions = dict()
for eartag in all_eartags:
    mouse = Mouse.from_db(eartag)
    mouse_details = ParticipantDetails.from_db(eartag, experiment_name)
    all_sessions = Session.list_all_sessions(mouse, experiment)
    pre_sr_sessions = [session for session in all_sessions if 'G4' not in session.session_dir]
    post_sr_sessions = [session for session in all_sessions if 'G4' in session.session_dir]
    session_timePoints_preSR = list()
    session_timePoints_postSR = list()
    for session in pre_sr_sessions:
        cut_folders = list(Path(session.session_dir).glob('*_cut/'))
        if len(cut_folders) > 0:
            selected_cut_folder = cut_folders.pop()
            trials_in_folders = list(Path(selected_cut_folder).glob('*'))
            selected_timepoint = trials_in_folders.pop()
        else:
            selected_timepoint = randrange(1,30)
        session_timePoints_preSR.append((session.session_dir, selected_timepoint))
    for session in post_sr_sessions:
        cut_folders = list(Path(session.session_dir).glob('*_cut/'))
        if len(cut_folders) > 0:
            selected_cut_folder = cut_folders.pop()
            trials_in_folders = list(Path(selected_cut_folder).glob('*'))
            selected_timepoint = trials_in_folders.pop()
        else:
            selected_timepoint = randrange(1,30)
        session_timePoints_postSR.append({session.session_dir: selected_timepoint})
    if len(post_sr_sessions) == 0:
        selected_timepoint = randrange(1,30)
        session_timePoints_postSR.append(('G4', selected_timepoint))
    all_selected_sessions[eartag] = {'preSR': session_timePoints_preSR, 'postSR': session_timePoints_postSR}

In [7]:
import pprint
pprint.pprint(all_selected_sessions)






{745: {'postSR': [{'/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20190507_CC1_G4': PosixPath('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20190507_CC1_G4/745_20190507_CC1_02_cut/745_20190507_CC1_02_V02.mp4')}],
       'preSR': [('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181130_CC1_G1',
                  PosixPath('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181130_CC1_G1/745_20181130_CC1_03_cut/745_20181130_CC1_03_V03.mp4')),
                 ('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181207_CC1_G2',
                  PosixPath('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181207_CC1_G2/745_20181207_CC1_01_cut/745_20181207_CC1_01_V05.mp4')),
                 ('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181214_CC2_G3',
                  PosixPath('/Volumes/SharedX/Neuro-Leventhal/data/mouseGrooming/et745/et745_20181214_CC2_G3/745_2018